# Michelin Restaurant Dataset: Preparation

The dataset comes from [Kaggle](https://www.kaggle.com/datasets/ngshiheng/michelin-guide-restaurants-2021)

In [0]:
import pandas as pd
import os
from pyspark import SparkFiles
from pyspark.sql.functions import *

### Catalog, Schema Creation

In [0]:
catalog_ = os.getenv('CATALOG_NAME')
schema_ = os.getenv('SCHEMA_NAME')
data_path_ = os.getenv('DATA_PATH')

In [0]:
spark.sql("CREATE CATALOG IF NOT EXISTS "+catalog_)
spark.sql("CREATE SCHEMA IF NOT EXISTS "+catalog_+"."+schema_)

In [0]:
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

### Data Ingestion from Volume [Bronze]

In [0]:
# Download the file to a volume
import urllib

volume_ = catalog_+'.'+schema_+'.init'
spark.sql("CREATE VOLUME IF NOT EXISTS "+volume_)
urllib.request.urlretrieve(
  data_path_,
  "/Volumes/"+catalog_+"/"+schema_+"/init/michelin_data.csv")
filepath_ = f"dbfs:/Volumes/"+catalog_+"/"+schema_+"/init/michelin_data.csv"


In [0]:
bronze_ = (spark.
           read.
           option("header", True).
           option("delimiter", ",").
           option("encoding", "utf-8").
           format("csv").
           load(filepath_).
           createOrReplaceTempView("bronze_v"))

display(spark.sql("SELECT * FROM bronze_v LIMIT 10;"))


In [0]:
%sql
DROP TABLE IF EXISTS bronze_data;
CREATE TABLE bronze_data AS


### Data Cleaning [Silver]